In [1]:
## OpenSearch 벡터 스토어 및 Amazon Bedrock을 사용한 RAG

In [2]:
!pip list | grep langchain
!pip list | grep opensearch

langchain                     0.0.345
langchain-core                0.0.9
langchain-experimental        0.0.42
opensearch-py                 2.3.2


In [3]:
import sys, os
module_path = ".."
sys.path.append(os.path.abspath(module_path))

## 1. Bedrock Client 생성

In [4]:
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

boto3_bedrock = bedrock.get_bedrock_client(
  assumed_role=None,
  endpoint_url=None,
  region="us-east-1"
)

Create new client
  Using region: us-east-1
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


## 2. Titan Embedding 및 LLM Claude-v2 모델 로딩
### claude-v2 model loading

In [5]:

from langchain.llms.bedrock import Bedrock
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# - create the Anthropic Model
llm_text = Bedrock(
    model_id=bedrock_info.get_model_id(model_name="Claude-V2"),
    client=boto3_bedrock,
    model_kwargs={
        "max_tokens_to_sample": 512
    },
    callbacks=[StreamingStdOutCallbackHandler()]
)
llm_text

Bedrock(client=<botocore.client.BedrockRuntime object at 0x7f71a45fb130>, model_id='anthropic.claude-v2', model_kwargs={'max_tokens_to_sample': 512}, callbacks=[<langchain_core.callbacks.streaming_stdout.StreamingStdOutCallbackHandler object at 0x7f71c8844250>])

### Embedding 모델 로딩

In [6]:
from langchain.embeddings.bedrock import BedrockEmbeddings
llm_emb = BedrockEmbeddings(
  model_id="amazon.titan-embed-text-v1",
  client=boto3_bedrock,
  region_name='us-east-1'
)

### 데이터 준비

In [7]:
import pandas as pd
pd.options.display.max_rows = 20

data_file_path = "data/fsi_smart_faq_ko.csv"
df = pd.read_csv(data_file_path)
df.head()

,no,Category,Information,type,Source
0,91,아마존 은행의 타기관OTP 이용등록방법 알려주세요,아마존 은행의 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 ...,인터넷뱅킹,아마존은행
1,90,아마존 공동인증서와 금융인증서 차이점이 무엇인가요?,공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자...,인증서,아마존은행
2,88,공동인증서와 금융인증서 차이점이 무엇인가요?,공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자...,인증서,신한은행
3,89,타기관OTP 이용등록방법 알려주세요,타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다....,인터넷뱅킹,신한은행
4,88,공동인증서와 금융인증서 차이점이 무엇인가요?,공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자...,인증서,신한은행


### 데이터 전처리
- column no 제거
- 기존 Category column name -> ask

In [8]:
pre_df = df.drop(columns=['no'], axis=1)
pre_df.rename(columns={"Category":"ask"}, inplace=True)
print_ww(pre_df.head())
pre_df.to_csv("data/fsi_smart_faq_ko_preprocess.csv", index=False)

                            ask  \
0   아마존 은행의 타기관OTP 이용등록방법 알려주세요
1  아마존 공동인증서와 금융인증서 차이점이 무엇인가요?
2      공동인증서와 금융인증서 차이점이 무엇인가요?
3           타기관OTP 이용등록방법 알려주세요
4      공동인증서와 금융인증서 차이점이 무엇인가요?

                                         Information   type Source
0  아마존 은행의 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 ...  인터넷뱅킹  아마존은행
1  공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자...    인증서  아마존은행
2  공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자...    인증서   신한은행
3  타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다....  인터넷뱅킹   신한은행
4  공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자...    인증서   신한은행


### CSV 문서 로딩

In [9]:
from langchain.document_loaders import CSVLoader

loader = CSVLoader(
  file_path="data/fsi_smart_faq_ko_preprocess.csv",
  source_column="Source",
  encoding="utf-8",
)

### 메타 데이터 생성
- 컬럼의 type, source는 metadata로 생성하고, 내용에서는 삭제
- 타임스탬프, 임베딩 모델의 endpoint 이름을 metadata로 추가

In [10]:
import time
documents_fsi = loader.load()

In [11]:
documents_fsi[0]

Document(page_content='ask: 아마존 은행의 타기관OTP 이용등록방법 알려주세요\nInformation: 아마존 은행의 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다. \r\n[경로]\r\n- 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록  \r\n- 아마존은행 쏠(SOL) 로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→   OTP이용등록\r\n \r\n ※ OTP이용등록후 재로그인을 하셔야 새로 등록된 보안매체가 적용됩니다.\r\n\r\n기타 궁금하신 내용은 아마존 은행 고객센터 1599-9999로 문의하여 주시기 바랍니다.\ntype: 인터넷뱅킹\nSource: 아마존은행', metadata={'source': '아마존은행', 'row': 0})

In [12]:
def create_metadata(docs):
  for idx, doc in enumerate(docs):
    # type 을 metadata로 저장
    split_content = doc.page_content.split("type: ")
    content = split_content[0]
    type = split_content[1].split("\n")[0]
    doc.page_content = content
    doc.metadata['type'] = type
    doc.metadata['timestamp'] = time.time()

create_metadata(documents_fsi)

In [13]:
documents_fsi[0]

Document(page_content='ask: 아마존 은행의 타기관OTP 이용등록방법 알려주세요\nInformation: 아마존 은행의 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다. \r\n[경로]\r\n- 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록  \r\n- 아마존은행 쏠(SOL) 로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→   OTP이용등록\r\n \r\n ※ OTP이용등록후 재로그인을 하셔야 새로 등록된 보안매체가 적용됩니다.\r\n\r\n기타 궁금하신 내용은 아마존 은행 고객센터 1599-9999로 문의하여 주시기 바랍니다.\n', metadata={'source': '아마존은행', 'row': 0, 'type': '인터넷뱅킹', 'timestamp': 1701758178.210838})

### Text Splitter 로 chunking

In [14]:
chunk_size = 2048
chunk_overlap = 0

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=chunk_size,
  chunk_overlap=chunk_overlap,
  separators=["\n\n", "\n", ".", " ", ""],
  length_function=len
)

docs = text_splitter.split_documents(documents_fsi)
print(f"Number of documents after split and chunking={len(docs)}")

Number of documents after split and chunking=92


## 4. OpenSearch Vector Indexer 생성

### Opensearch id/passwd

In [15]:
aws_region = "ap-northeast-2"
opensearch_domain_endpoint = "https://search-bedrock-fkik524xhgkrn3t2omdtfspzmm.ap-northeast-2.es.amazonaws.com"

http_auth = ("admin", "Admin123@")
index_name = "genai-demo-index-v1"

### 오픈서치 인덱스 유무에 따라 삭제
오픈서치에 해당 인덱스가 존재하면 삭제


In [16]:
from utils.opensearch import opensearch_utils

In [ ]:
os_client = opensearch_utils.create_aws_opensearch_client(
  aws_region,
  opensearch_domain_endpoint,
  http_auth=http_auth
)

index_exist = opensearch_utils.check_if_index_exists(
  os_client,
  index_name
)

if index_exist:
  opensearch_utils.delete_index(
    os_client,
    index_name
  )
else:
  print("index does not exist.")

In [ ]:
from langchain.vectorstores import OpenSearchVectorSearch

In [47]:
docsearch = OpenSearchVectorSearch.from_documents(
  index_name=index_name,
  documents=docs,
  embedding=llm_emb,
  opensearch_url=opensearch_domain_endpoint,
  http_auth=http_auth,
  bulk_size=10000,
  timeout=60
)

In [52]:
vector_db = OpenSearchVectorSearch(
  index_name=index_name,
  opensearch_url=opensearch_domain_endpoint,
  embedding_function=llm_emb,
  http_auth=http_auth,
  is_aoss=False,
  space_type="l2"
)

## 5. 오픈서치에 "유사 서치" 검색